<a href="https://colab.research.google.com/github/parker-group/Kinshasa_EO/blob/main/GEE_MODIS_LST_Export_Kinshasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MODIS LST Export - Colab Version
!pip install earthengine-api geemap --quiet

import ee
import geemap
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

ee.Authenticate()
ee.Initialize(project='ee-salempar')

roi = ee.FeatureCollection("projects/ee-salempar/assets/Kinshasa_AiresSelect2")

start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 12, 31)

modis_lst = (
    ee.ImageCollection('MODIS/061/MOD11A2')
    .filterBounds(roi)
    .filterDate(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
    .select('LST_Day_1km')
    .map(lambda img: img
         .multiply(0.02)
         .subtract(273.15)
         .rename('LST_C')
         .copyProperties(img, ['system:time_start']))
)

def export_monthly_modis_lst(collection, band_name, label_prefix, scale, folder, reducer):
    current = start_date
    while current <= end_date:
        next_month = current + relativedelta(months=1)
        label = current.strftime("%Y_%m")
        print(f"\U0001F4C6 Processing: {label}")

        ee_start = ee.Date(current.strftime("%Y-%m-%d"))
        ee_end = ee.Date(next_month.strftime("%Y-%m-%d"))

        image = collection.filterDate(ee_start, ee_end).select(band_name).reduce(reducer)
        image = ee.Image(image).clip(roi)

        try:
            if image.bandNames().size().getInfo() > 0:
                task = ee.batch.Export.image.toDrive(**{
                    'image': image,
                    'description': f"{label_prefix}_{label}",
                    'folder': f"{folder}_{label_prefix}",
                    'fileNamePrefix': f"{label_prefix}_{label}",
                    'scale': scale,
                    'region': roi.geometry(),
                    'crs': 'EPSG:4326',
                    'maxPixels': 1e13
                })
                task.start()
                print(f"✅ Started export: {label_prefix}_{label}")
                time.sleep(2)
            else:
                print(f"⚠️ Skipped: {label_prefix}_{label} — no valid data")
        except Exception as e:
            print(f"❌ Error on {label_prefix}_{label}: {str(e)}")

        current = next_month

export_monthly_modis_lst(
    collection=modis_lst,
    band_name='LST_C',
    label_prefix='MODIS_LST',
    scale=1000,
    folder='GEE_Kinshasa',
    reducer=ee.Reducer.mean()
)


📆 Processing: 2022_01
✅ Started export: MODIS_LST_2022_01
📆 Processing: 2022_02
✅ Started export: MODIS_LST_2022_02
📆 Processing: 2022_03
✅ Started export: MODIS_LST_2022_03
📆 Processing: 2022_04
✅ Started export: MODIS_LST_2022_04
📆 Processing: 2022_05
✅ Started export: MODIS_LST_2022_05
📆 Processing: 2022_06
✅ Started export: MODIS_LST_2022_06
📆 Processing: 2022_07
✅ Started export: MODIS_LST_2022_07
📆 Processing: 2022_08
✅ Started export: MODIS_LST_2022_08
📆 Processing: 2022_09
✅ Started export: MODIS_LST_2022_09
📆 Processing: 2022_10
✅ Started export: MODIS_LST_2022_10
📆 Processing: 2022_11
✅ Started export: MODIS_LST_2022_11
📆 Processing: 2022_12
✅ Started export: MODIS_LST_2022_12
📆 Processing: 2023_01
✅ Started export: MODIS_LST_2023_01
📆 Processing: 2023_02
✅ Started export: MODIS_LST_2023_02
📆 Processing: 2023_03
✅ Started export: MODIS_LST_2023_03
📆 Processing: 2023_04
✅ Started export: MODIS_LST_2023_04
📆 Processing: 2023_05
✅ Started export: MODIS_LST_2023_05
📆 Processing: 